# Convolutional Neural Network

Necessary steps to conduct preprocessing, and then define, train, and test a convolutional neural network model on image data.

## Resources

The below links contain interesting information related to classify the Fashion-MNIST dataset:
* https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a
* https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb
* https://www.kaggle.com/anebzt/fashion-mnist-in-keras
* https://machinelearningmastery.com/how-to-perform-object-detection-with-yolov3-in-keras/
* https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a
* https://github.com/SocieteGenevoiseDonnees/FiltersStylesDomains

## Objects Detection Systems

* [Yolo3](https://pjreddie.com/darknet/yolo/)

## Sample Example

The data consists of images of clothing.

Now, lets begin by loading some of the required packages:

In [1]:
import numpy as np

import pandas as pd
from pandas import read_csv

import matplotlib.pyplot as plt

import keras

from keras.utils import to_categorical
# Eithe Keras or Sklearn function can be used for one-hot encodong
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

%matplotlib inline


import warnings                      # Following section is throwing a tone of warnings due to 
warnings.filterwarnings('ignore')    # versions mismatch in current system, therefore ignoring

Using TensorFlow backend.


### Load the Dataset

The two distinct datasets are loaded; one for training and one for testing. These are stored in the corresponding training and testing sets.

In [3]:
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

A dictionary key is defined below with the corresponding class name for each of the integer encoded values in y_train and y_test.

In [4]:
key = {0:"T-shirt/top",1:"Trouser",2:"Pullover",3:"Dress",4:"Coat",
       5:"Sandal",6:"Shirt",7:"Sneaker",8:"Bag",9:"Ankle boot"}

## Applying and Training Model

### Datasets

* Exhaustive List: https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research#Object_detection_and_recognition
* ImageNet: http://www.image-net.org/
* Downsampled ImageNet: http://image-net.org/small/download.php
* CIFAR: https://www.cs.toronto.edu/~kriz/cifar.html


### Resources

* https://medium.com/coinmonks/how-to-get-images-from-imagenet-with-python-in-google-colaboratory-aeef5c1c45e5
* https://blog.goodaudience.com/train-a-keras-neural-network-with-imagenet-synsets-in-google-colaboratory-e68dc4fd759f
* https://www.pyimagesearch.com/2016/08/10/imagenet-classification-with-python-and-keras/

### Process

The process to train model from external datasets is to download the dataset and place in corresponding directories before performing the trainng itself.

Another potential solution is to use a pretrained model.

## Transfer Learning

In [8]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
import zipfile

# Extract zip archive
with zipfile.ZipFile('cats_dogs_tlearn_basic_cnn.h5.zip', 'r') as zip_ref:
    zip_ref.extractall()

# load model
model = load_model('cats_dogs_tlearn_basic_cnn.h5')
# summarize model.
model.summary()

OSError: Unable to open file (unable to open file: name = 'cats_dogs_tlearn_basic_cnn.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)